<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://nbviewer.org/github/ycseong07/SI_ML/blob/main/3_modeling_oversampling_v1.3.ipynb"><img src="https://jupyter.org/assets/share.png" width="60" />주피터 노트북 뷰어로 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ycseong07/SI_ML/blob/main/3_modeling_oversampling_v1.3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
</table>

In [2]:
!pip install aif360
!pip install shap

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import joblib
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score, roc_curve, precision_recall_curve

from aif360.sklearn.metrics import specificity_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from imblearn.over_sampling import *
from sklearn import metrics
import pickle
import shap

     |████████████████████████████████| 175 kB 8.1 MB/s 
     |████████████████████████████████| 564 kB 50.0 MB/s 
  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31284 sha256=98ae3e59d59e2bd2d631835b8dad37602ca32c3e75be8ddf1e08e60da3343588
  Stored in directory: /root/.cache/pip/wheels/67/2b/fb/326e30d638c538e69a5eb0aa47f4223d979f502bbdb403950f
Successfully built memory-profiler


# Data load

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# train_2014 = pd.read_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2014_NSOK_train_preprocessed_2.csv')
# train_2014 = train_2014.drop('Unnamed: 0', axis=1)

# test_2014 = pd.read_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2014_NSOK_test_preprocessed_2.csv')
# test_2014 = test_2014.drop('Unnamed: 0', axis=1)

# test_2017 = pd.read_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2017_NSOK_preprocessed_2.csv')
# test_2017 = test_2017.drop('Unnamed: 0', axis=1)

# test_2020 = pd.read_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2020_NSOK_preprocessed_2.csv')
# test_2020 = test_2020.drop('Unnamed: 0', axis=1)

In [ ]:
# cor_matrix = train_2014.corr().abs()

# upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))

# to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
# print(); print(to_drop)

# ## cohabchild_financial_giving', 'E1_3_1_y_1 제거

# train_2014 = train_2014.drop(train_2014[to_drop], axis=1)
# test_2014 = test_2014.drop(test_2014[to_drop], axis=1)
# test_2017 = test_2017.drop(test_2017[to_drop], axis=1)
# test_2020 = test_2020.drop(test_2020[to_drop], axis=1)
# print(); print(train_2014.head())


['cohabchild_financial_giving', 'E1_3_1_y_1']

   H12_1_etc_y  financial_asset  B2_2R_2_y  ...  E1_2_y_4  G2_3_1_y  suicide
0     0.000000         0.008395          0  ...         0         1      0.0
1     0.062500         0.031649          0  ...         0         0      0.0
2     0.000000         0.065061          0  ...         0         1      0.0
3     0.000000         0.000525          0  ...         0         1      0.0
4     0.041667         0.025185          1  ...         0         1      0.0

[5 rows x 177 columns]


In [ ]:
# print(train_2014.shape)
# print(test_2014.shape)
# print(test_2017.shape)
# print(test_2020.shape)

(7196, 177)
(3085, 177)
(10083, 177)
(9920, 177)


In [ ]:
# len(train_2014[train_2014['suicide']==1])

701

In [ ]:
# ## 4. over sampling for balanced data

# smote = ADASYN(random_state=1120)
# # smote = SMOTE()

# X_train_df = train_2014.drop('suicide', axis=1)
# y_train_df = train_2014['suicide']

# X_train, y_train = smote.fit_resample(X_train_df, y_train_df)

# X_train = pd.DataFrame(X_train, columns=X_train_df.columns)
# y_train = pd.DataFrame(y_train)

In [ ]:
# sample_over = pd.concat([X_train, y_train], axis=1)
# sample_over.shape

(12940, 177)

In [ ]:
# sample_over.to_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/train_over_adasyn.csv')
# test_2014.to_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2014_NSOK_test_preprocessed_2.csv')
# test_2017.to_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2017_NSOK_test_preprocessed_2.csv')
# test_2020.to_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2020_NSOK_test_preprocessed_2.csv')

## 여기부터

In [3]:
sample_over = pd.read_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/train_over_adasyn.csv')
sample_over = sample_over.drop('Unnamed: 0', axis=1)

test_2014 = pd.read_csv('/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2014_NSOK_test_preprocessed_2.csv')
test_2014 = test_2014.drop('Unnamed: 0', axis=1)

In [4]:
print(sample_over.shape)
print(test_2014.shape)

(12940, 177)
(3085, 177)


In [5]:
X_train = sample_over.drop('suicide', axis=1)
y_train = sample_over['suicide']

X_test = test_2014.drop('suicide', axis=1)
y_test = test_2014['suicide']

In [6]:
y_train.isnull().sum()[y_train.isnull().sum()!=0]

array([], dtype=int64)

In [7]:
X_train.shape

(12940, 176)

# Make 5 ML models

In [8]:
models = []
params = []

## 1. Logistic model

In [ ]:
model = ('LR', LogisticRegression(solver='liblinear', multi_class='auto', random_state=1120))
param = {
    'C': [i/10.0 for i in range(1, 10)],
    'class_weight': [None],
    'penalty': ['l2']
}

models.append(model)
params.append(param)

## l2 적용 가능
# {'C': 1, 'class_weight': 'balanced', 'penalty': 'l2'}

## 2. Random Forest model
https://injo.tistory.com/30



In [ ]:

model = ('RF', RandomForestClassifier(random_state=1120))
param = {
    'n_estimators': [500],
    'min_samples_split': [2, 5, 10, 30],
    'min_samples_leaf': [2, 5, 10, 30],
    'max_features': [20,35,50,65],
    'max_depth': [20, 30, 40, 45],
    'class_weight': [None]
}

models.append(model)
params.append(param)

# {'class_weight': None, 'max_depth': 10, 'max_features': 72, 'min_samples_leaf': 150, 'min_samples_split': 150, 'n_estimators': 500}
# {'class_weight': None, 'max_depth': 13, 'max_features': 90, 'min_samples_leaf': 50, 'min_samples_split': 50, 'n_estimators': 500}
# {'class_weight': None, 'max_depth': 13, 'max_features': 90, 'min_samples_leaf': 10, 'min_samples_split': 10, 'n_estimators': 500}
# {'class_weight': None, 'max_depth': 13, 'max_features': 90, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 500}
# {'class_weight': None, 'max_depth': 13, 'max_features': 90, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}
# {'class_weight': None, 'max_depth': 13, 'max_features': 80, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}
# {'class_weight': None, 'max_depth': 15, 'max_features': 60, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}
# {'class_weight': None, 'max_depth': 19, 'max_features': 30, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}
# {'class_weight': None, 'max_depth': 50, 'max_features': 40, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}
# {'class_weight': None, 'max_depth': 50, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}
# {'class_weight': None, 'max_depth': 45, 'max_features': 40, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}
# {'class_weight': None, 'max_depth': 45, 'max_features': 40, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}

    # 'n_estimators': [500],
    # 'min_samples_split': [300, 500, 700, 900],
    # 'min_samples_leaf': [300, 500, 700, 900],
    # 'max_features': [50, 60, 70, 80],
    # 'max_depth': [10, 13, 16],
    # 'class_weight': [None]

## L2???
# To use L2 regularization, you need a vector to take the norm of. For example, in linear regression or linear classification, we use the norm of the weight vector to make sure none of the weights gets too large, and a lot of weights will be set to 0 (due to the nature of the  1 -norm).
# What vector would you use for random forests? In the standard random forest classifier there is no suitable weight vector.
# The only thing I can think of is to give each tree a weight (the default is that each tree gets an equal weight) but using an L1 norm would make it so that a lot of the trees in the forest get no vote — this goes counter to the entire philosophy behind the method

## 3. Feed-forward Network (MLP)

In [ ]:
model = ('MLP', MLPClassifier(random_state=1120, max_iter=2000, batch_size=128))
param = {
    'hidden_layer_sizes': [(5), (10), (15), (20)],
    'alpha': [0, 0.1],
    'learning_rate': ['adaptive'],
    'activation': ['logistic'],
    'solver': ['adam']
}

models.append(model)
params.append(param)

# {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': 20, 'learning_rate': 'adaptive', 'solver': 'sgd'}
# {'activation': 'logistic', 'alpha': 0.05, 'hidden_layer_sizes': 15, 'learning_rate': 'adaptive', 'solver': 'adam'}
# {'activation': 'logistic', 'alpha': 0.05, 'hidden_layer_sizes': 5, 'learning_rate': 'adaptive', 'solver': 'adam'}
# {'activation': 'logistic', 'alpha': 0.05, 'hidden_layer_sizes': 5, 'learning_rate': 'adaptive', 'solver': 'adam'}
# {'activation': 'logistic', 'alpha': 0.04, 'hidden_layer_sizes': 5, 'learning_rate': 'adaptive', 'solver': 'adam'}

## l2 reg -> alpha 값으로 조절

## 4. Make SVM model

In [ ]:
model = ('SVC', SVC(random_state=1120, probability=True))
param = [
    # {'kernel': ['linear'], 'C': [3,5,10, 20, 50], 'gamma': [0.1]},
    # {'kernel': ['rbf'], 'C': [0.9], 'gamma': [0.02]},
    # {'kernel': ['poly'], 'C': [1], 'degree': [1], 'gamma': [0.3]},
    {'kernel': ['sigmoid'], 'C': [0.6], 'degree': [0.1], 'gamma': [0.01]}
]

models.append(model)
params.append(param)

#{'C': 5, 'gamma': 0.1, 'kernel': 'linear'}

# {'C': 0.6, 'degree': 0.1, 'gamma': 0.01, 'kernel': 'sigmoid'}

## l2 는 linear에만 적용가능
# from sklearn import svm
# svc = svm.LinearSVC(C=1, loss='hinge', max_iter=1000)
# svc

## 5. Make XGBoost model

https://injo.tistory.com/44

In [ ]:
# model = ('XGB', XGBClassifier(random_state=1120, use_label_encoder=False, tree_method='gpu_hist')) ## gpu_hist 옵션주면 shap 돌릴 때 런타임 사망..
model = ('XGB', XGBClassifier(random_state=1120, tree_method='gpu_hist')) ## gpu_hist 옵션주면 shap 돌릴 때 런타임 사망..
# model = ('XGB', XGBClassifier(random_state=1120, use_label_encoder=False))
param = {
    'learning_rate': [0.01], ## 오버피팅 막기위해 learning_rate을 낮추면 n_estimators는 높여주어야 함
    'n_estimators': [3000],
    'min_child_weight': [0], ## 클수록 오버피팅 방지
    'max_depth': [13], ## 작을수록
    'subsample':[0.55],
    'colsample_bytree':[0.9],
    'scale_pos_weight' : [1],
    'gamma':[0],
    'reg_alpha':[1e-5]
}

models.append(model)
params.append(param)

## 이것도 tree기반이니까 안될듯. 

#{'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 1000, 'scale_pos_weight': 1, 'subsample': 0.7}
# {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 11, 'min_child_weight': 8, 'n_estimators': 1000, 'scale_pos_weight': 1, 'subsample': 0.7}
# {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 20, 'min_child_weight': 7, 'n_estimators': 1000, 'scale_pos_weight': 1, 'subsample': 0.7}
# {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 15, 'min_child_weight': 5, 'n_estimators': 1000, 'scale_pos_weight': 1, 'subsample': 0.7}
# {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 15, 'min_child_weight': 2, 'n_estimators': 1000, 'scale_pos_weight': 1, 'subsample': 0.7}
# {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 19, 'min_child_weight': 0, 'n_estimators': 1000, 'scale_pos_weight': 1, 'subsample': 0.7} --overfitting
# {'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 19, 'min_child_weight': 0, 'n_estimators': 1000, 'scale_pos_weight': 1, 'subsample': 0.6}
# {'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 19, 'min_child_weight': 0, 'n_estimators': 1000, 'scale_pos_weight': 1, 'subsample': 0.5}
# {'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 19, 'min_child_weight': 0, 'n_estimators': 1000, 'reg_alpha': 1e-05, 'scale_pos_weight': 1, 'subsample': 0.5}
# https://www.kaggle.com/lifesailor/xgboost

In [ ]:
model[1]

XGBClassifier(random_state=1120, tree_method='gpu_hist')

# Grid search & Fitting the model

In [ ]:
## scoring = roc_auc, f1, recall ... 
def fitting_cv(model, param, kfold, train_input, train_target, scoring='roc_auc', n_jobs=1, tracking=True, return_train_score=True):
    name = model[0]
    estimator = model[1]
    if tracking:
        start_time = datetime.now()
        print("[%s] Start parameter search for model '%s'" % (start_time, name))
        gridsearch = GridSearchCV(estimator=estimator, param_grid=param, cv=cv_idx, scoring=scoring, n_jobs=n_jobs, return_train_score=True)
        gridsearch.fit(train_input, train_target)
        end_time = datetime.now()
        duration_time = (end_time - start_time).seconds
        print("[%s] Finish parameter search for model '%s' (time: %d seconds)" % (end_time, name, duration_time))
        print()
    else:
        gridsearch = GridSearchCV(estimator=estimator, param_grid=param, cv=kfold, scoring=scoring, n_jobs=n_jobs)
        gridsearch.fit(train_input, train_target)
    
    return gridsearch   

In [ ]:
# Generating cross-validation index
cv_idx = StratifiedKFold(n_splits=10, shuffle=True, random_state=1120)

In [ ]:
result = fitting_cv(model=model, param=param, kfold=cv_idx, train_input=X_train.values, train_target=y_train.values, n_jobs=-1)

In [ ]:
# # plot learning curve
# from numpy import loadtxt
# from xgboost import XGBClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from matplotlib import pyplot


# # fit model no training data
# model = XGBClassifier(random_state=1120, 
#     eval_metric=["error", "auc"],
#     learning_rate = 0.01,
#     n_estimators = 3000,
#     min_child_weight = 0, 
#     max_depth = 13,
#     subsample = 0.55,
#     colsample_bytree = 0.9,
#     scale_pos_weight = 1,
#     gamma = 0,
#     reg_alpha = 1e-5
#     )
# XGBClassifier()



# eval_set = [(X_train, y_train), (X_test, y_test)]
# model.fit(X_train, y_train, eval_metric=["error", "logloss"], eval_set=eval_set, verbose=True)
# # make predictions for test data
# y_pred = model.predict(X_test)
# predictions = [round(value) for value in y_pred]
# # evaluate predictions
# accuracy = accuracy_score(y_test, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))
# # retrieve performance metrics
# results = model.evals_result()
# epochs = len(results['validation_0']['error'])
# x_axis = range(0, epochs)
# # plot log loss
# fig, ax = pyplot.subplots()
# ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
# ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
# ax.legend()
# pyplot.ylabel('Log Loss')
# pyplot.title('XGBoost Log Loss')
# pyplot.show()
# # plot classification error
# fig, ax = pyplot.subplots()
# ax.plot(x_axis, results['validation_0']['error'], label='Train')
# ax.plot(x_axis, results['validation_1']['error'], label='Test')
# ax.legend()
# pyplot.ylabel('Classification Error')
# pyplot.title('XGBoost Classification Error')
# pyplot.show()

In [ ]:
print(result.cv_results_)

In [97]:
# model_file_path = "/content/gdrive/My Drive/성은이파이썬/ICT_ML/model/2014_over_model_LR.model"  
# result = joblib.load(model_file_path)

# model_file_path = "/content/gdrive/My Drive/성은이파이썬/ICT_ML/model/2014_over_model_RF.model"  
# result = joblib.load(model_file_path)

# model_file_path = "/content/gdrive/My Drive/성은이파이썬/ICT_ML/model/2014_over_model_MLP.model"  
# result = joblib.load(model_file_path)

# model_file_path = "/content/gdrive/My Drive/성은이파이썬/ICT_ML/model/2014_over_model_SVC.model"  
# result = joblib.load(model_file_path)

model_file_path = "/content/gdrive/My Drive/성은이파이썬/ICT_ML/model/2014_over_model_XGB.model"  
result = joblib.load(model_file_path)

In [98]:
print(result.best_params_)

{'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 13, 'min_child_weight': 0, 'n_estimators': 3000, 'reg_alpha': 1e-05, 'scale_pos_weight': 1, 'subsample': 0.56}


In [99]:
# ## check over fitting
# test_scores = result.cv_results_['mean_test_score']
# train_scores = result.cv_results_['mean_train_score'] 

# plt.plot(test_scores, label='test')
# plt.plot(train_scores, label='train')
# plt.legend(loc='best')
# plt.show()

In [100]:
y_train_prob = result.predict_proba(X_train.values)
y_train_pred = result.predict(X_train.values)

print('Accuracy: %.4f' % metrics.accuracy_score(y_train, y_train_pred))
print('AUC: %.4f' % metrics.roc_auc_score(y_train, y_train_prob[:, 1]))
print(metrics.classification_report(y_train, y_train_pred))

Accuracy: 1.0000
AUC: 1.0000
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6495
         1.0       1.00      1.00      1.00      6445

    accuracy                           1.00     12940
   macro avg       1.00      1.00      1.00     12940
weighted avg       1.00      1.00      1.00     12940



In [101]:
# https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/
# Optimal Threshold for Precision-Recall Curve (maximizing f1-score)

y_test_prob = result.predict_proba(X_test.values)

# calculate pr-curve
precision, recall, thresholds = precision_recall_curve(y_test, y_test_prob[:,1])

# convert to f score
fscore = (2 * precision * recall) / (precision + recall)

# locate the index of the largest f score
ix = np.nanargmax(fscore)
print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))

Best Threshold=0.377903, F-Score=0.335


In [102]:
y_test_prob = result.predict_proba(X_test.values)
y_test_pred  = (y_test_prob[:,1] >= thresholds[ix]).astype(bool)
# y_test_pred  = result.predict(X_test.values)

In [103]:
print('Accuracy: %.4f' % metrics.accuracy_score(y_test, y_test_pred))
print('AUC: %.4f' % metrics.roc_auc_score(y_test, y_test_prob[:, 1]))
print(metrics.classification_report(y_test, y_test_pred))

Accuracy: 0.8454
AUC: 0.7453
              precision    recall  f1-score   support

         0.0       0.93      0.89      0.91      2785
         1.0       0.29      0.40      0.33       300

    accuracy                           0.85      3085
   macro avg       0.61      0.65      0.62      3085
weighted avg       0.87      0.85      0.86      3085



In [104]:
## AUROC
print('%.4f' % metrics.roc_auc_score(y_test, y_test_prob[:, 1]))

## AUPRC
print('%.4f' % metrics.average_precision_score(y_test, y_test_prob[:, 1]))

# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_test_pred)
print('%.4f' % f1)

# cohen's kappa score
kappa = cohen_kappa_score(y_test, y_test_pred)
print('%.4f' % kappa)

# Sensitivity: tp / (tp + fn)
recall = recall_score(y_test, y_test_pred)
print('%.4f' % recall)

## acc
print('%.4f' % metrics.accuracy_score(y_test, y_test_pred))

# Specificity 
specificity = specificity_score(y_test, y_test_pred)
print('%.4f' % specificity)

# precision tp / (tp + fp)
precision = precision_score(y_test, y_test_pred)
print('%.4f' % precision)

# Threshold
print('%.4f' % thresholds[ix])

0.7453
0.2695
0.3347
0.2499
0.4000
0.8454
0.8934
0.2878
0.3779


In [105]:
metrics.confusion_matrix(y_test, y_test_pred, labels=None, sample_weight=None, normalize=None)

array([[2488,  297],
       [ 180,  120]])

In [106]:
# # fn for optimal threshold search
# def expect_f1(proba, thres, y):
#     lr_pred_new = [1 if proba[i]> thres else 0 for i in range(len(proba))]
#     return metrics.f1_score(y, lr_pred_new)

## External validation 2017

In [107]:
data_path = '/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2017_NSOK_test_preprocessed_2.csv'

ex_val_data = pd.read_csv(data_path) ## sperater, delimiter 확인
ex_val_data = ex_val_data.drop('Unnamed: 0', axis=1)

X_ex_val = ex_val_data.drop('suicide', axis=1)
y_ex_val = ex_val_data['suicide']

In [108]:
y_ex_val_prob = result.predict_proba(X_ex_val.values)

precision, recall, thresholds = precision_recall_curve(y_ex_val, y_ex_val_prob[:,1])
fscore = (2 * precision * recall) / (precision + recall)
ix = np.nanargmax(fscore)
print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))

y_ex_val_pred  = (y_ex_val_prob[:,1] >= thresholds[ix]).astype(bool)

Best Threshold=0.495918, F-Score=0.290


invalid value encountered in true_divide


In [109]:
print('Accuracy: %.4f' % metrics.accuracy_score(y_ex_val, y_ex_val_pred))
print('AUC: %.4f' % metrics.roc_auc_score(y_ex_val, y_ex_val_prob[:, 1]))
print(metrics.classification_report(y_ex_val, y_ex_val_pred))

Accuracy: 0.8990
AUC: 0.7536
              precision    recall  f1-score   support

         0.0       0.95      0.94      0.95      9453
         1.0       0.26      0.33      0.29       630

    accuracy                           0.90     10083
   macro avg       0.61      0.63      0.62     10083
weighted avg       0.91      0.90      0.90     10083



In [110]:
## AUROC
print('%.4f' % metrics.roc_auc_score(y_ex_val, y_ex_val_prob[:, 1]))

## AUPRC
print('%.4f' % metrics.average_precision_score(y_ex_val, y_ex_val_prob[:, 1]))

# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_ex_val, y_ex_val_pred)
print('%.4f' % f1)

# cohen's kappa score
kappa = cohen_kappa_score(y_ex_val, y_ex_val_pred)
print('%.4f' % kappa)

# Sensitivity: tp / (tp + fn)
recall = recall_score(y_ex_val, y_ex_val_pred)
print('%.4f' % recall)

## acc
print('%.4f' % metrics.accuracy_score(y_ex_val, y_ex_val_pred))

# Specificity 
specificity = specificity_score(y_ex_val, y_ex_val_pred)
print('%.4f' % specificity)

# precision tp / (tp + fp)
precision = precision_score(y_ex_val, y_ex_val_pred)
print('%.4f' % precision)

# Threshold
print('%.4f' % thresholds[ix])

0.7536
0.1800
0.2901
0.2366
0.3302
0.8990
0.9370
0.2587
0.4959


In [111]:
metrics.confusion_matrix(y_ex_val, y_ex_val_pred, labels=None, sample_weight=None, normalize=None)

array([[8857,  596],
       [ 422,  208]])

## External validation 2020

In [112]:
data_path = '/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2020_NSOK_test_preprocessed_2.csv'

ex_val_data = pd.read_csv(data_path) ## sperater, delimiter 확인
ex_val_data = ex_val_data.drop('Unnamed: 0', axis=1)


X_ex_val = ex_val_data.drop('suicide', axis=1)
y_ex_val = ex_val_data['suicide']

In [113]:
# Optimal Threshold for ROC Curve (maximizing f1)

y_ex_val_prob = result.predict_proba(X_ex_val.values)

precision, recall, thresholds = precision_recall_curve(y_ex_val, y_ex_val_prob[:,1])
fscore = (2 * precision * recall) / (precision + recall)
ix = np.nanargmax(fscore)
print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))

y_ex_val_pred  = (y_ex_val_prob[:,1] >= thresholds[ix]).astype(bool)

Best Threshold=0.430465, F-Score=0.172


In [114]:
# y_train_prob = result.predict_proba(X_train.values)
# y_train_pred = result.predict(X_train.values)

# y_ex_val_prob = result.predict_proba(X_ex_val.values)
# y_ex_val_pred  = (y_ex_val_prob[:,1] >= thresholds[ix]).astype(bool)
# y_ex_val_pred  = result.predict(X_ex_val.values) ###

In [115]:
print('Accuracy: %.4f' % metrics.accuracy_score(y_ex_val, y_ex_val_pred))
print('AUC: %.4f' % metrics.roc_auc_score(y_ex_val, y_ex_val_prob[:, 1]))
print(metrics.classification_report(y_ex_val, y_ex_val_pred))

Accuracy: 0.9535
AUC: 0.7764
              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98      9733
         1.0       0.13      0.26      0.17       187

    accuracy                           0.95      9920
   macro avg       0.56      0.61      0.57      9920
weighted avg       0.97      0.95      0.96      9920



In [116]:
## AUROC
print('%.4f' % metrics.roc_auc_score(y_ex_val, y_ex_val_prob[:, 1]))

## AUPRC
print('%.4f' % metrics.average_precision_score(y_ex_val, y_ex_val_prob[:, 1]))

# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_ex_val, y_ex_val_pred)
print('%.4f' % f1)

# cohen's kappa score
kappa = cohen_kappa_score(y_ex_val, y_ex_val_pred)
print('%.4f' % kappa)

# Sensitivity: tp / (tp + fn)
recall = recall_score(y_ex_val, y_ex_val_pred)
print('%.4f' % recall)

## acc
print('%.4f' % metrics.accuracy_score(y_ex_val, y_ex_val_pred))

# Specificity 
specificity = specificity_score(y_ex_val, y_ex_val_pred)
print('%.4f' % specificity)

# precision tp / (tp + fp)
precision = precision_score(y_ex_val, y_ex_val_pred)
print('%.4f' % precision)

# Threshold
print('%.4f' % thresholds[ix])

0.7764
0.0833
0.1724
0.1511
0.2567
0.9535
0.9669
0.1297
0.4305


In [117]:
metrics.confusion_matrix(y_ex_val, y_ex_val_pred, labels=None, sample_weight=None, normalize=None)

array([[9411,  322],
       [ 139,   48]])

In [ ]:
# save model
# model_file_path = "/content/gdrive/My Drive/성은이파이썬/ICT_ML/model/2014_over_model_%s.model" % model[0] 
# joblib.dump(result, model_file_path)

['/content/gdrive/My Drive/성은이파이썬/ICT_ML/model/2014_over_model_LR.model']

# Visualize model's results

In [ ]:
model_file_path = "/content/gdrive/My Drive/성은이파이썬/ICT_ML/model/2014_over_model_XGB.model"  
result = joblib.load(model_file_path)

Trying to unpickle estimator LabelEncoder from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
Trying to unpickle estimator GridSearchCV from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations


In [ ]:
# model = pickle.load(open("saved_model_file", "rb"))
# cols_when_model_builds = model.get_booster().feature_names
# pd_dataframe = pd_dataframe[cols_when_model_builds]

## 2. SHAP
- https://github.com/slundberg/shap

In [ ]:
best_model = result.best_estimator_
explainer = shap.TreeExplainer(best_model)

In [ ]:
# col_dict = {'performance' : 'Physical malperform',
# 'RES_AGE_y' : 'Age',
# 'living_sudo' : 'Living in metropolitan area',
# 'J1b_1_y' : 'Expense for medical treatment',
# 'gds15_i' : 'Depression (gds15)',
# 'H16_7_y' : 'Life satisfaction',
# 'bmi' : 'BMI',
# 'cohab_mean_age' : 'Mean age of cohabitants',
# 'friends_contact' : 'Frequency of contacting with friends',
# 'B2_3_y' : 'Number of chronic diseases',
# 'J6b_1_y' : 'Real estate property',
# 'J1b_4_y' : 'Expense for events (e.g., funeral)',
# 'H16_4_y' : 'Relation satisfaction with children',
# 'call_child_out' : 'Frequency of phone call with seperated children',
# 'cohab_mean_edu_y' : 'Mean educational year of cohabitants',
# 'H14_1_3_y' : 'Experience of feeling offended by someone',
# 'J6b_2_y' : 'Financial property',
# 'J3b_3_3_y' : 'Family income',
# 'E2_2_1_y' : 'Length of prior work experience',
# 'B4_1_etc_y' : 'Number of hospital visits',
# 'H16_3_y' : 'Relation satisfaction with spouse',
# 'J6b_4_y' : 'Financial debt',
# 'J3b_3_13_y' : 'family total income',
# 'G4_3_6_y' : 'Helping spouse for medical purpose', ## 도움 줌
# 'H16_2_y' : 'Subjective economic satisfaction',
# 'num_alter' : 'Number of friends',
# 'D8_y_0' : 'Having no religion',
# 'Q1_y_1': 'Living in a detached house',
# 'nutrition' : 'Malnutrition',
# 'B3_y':'The number of drugs taking for more than 3 months'
# }


X_test_r = X_test.rename(columns=col_dict)
df = X_test_r.sample(n=1000, random_state = 1120)

In [ ]:
shap_values = explainer.shap_values(df,check_additivity=False)

In [ ]:
shap.summary_plot(shap_values, df, max_display=20)

In [ ]:
shap.summary_plot(shap_values, df, plot_type="bar", max_display=20)

In [ ]:
# data_path = '/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2017_NSOK_preprocessed_2.csv'

# ex_val_data = pd.read_csv(data_path) ## sperater, delimiter 확인
# ex_val_data = ex_val_data.drop('Unnamed: 0', axis=1)
ex_val_data = test_2017

X_ex_val = ex_val_data.drop('suicide', axis=1)
y_ex_val = ex_val_data['suicide']

In [ ]:
# col_dict = {'performance' : 'Physical malperform',
# 'gds15_i' : 'Depression (gds15)',
# 'living_sudo' : 'Living in metropolitan area',
# 'RES_AGE_y' : 'Age',
# 'H16_7_y' : 'Life satisfaction',
# 'H14_1_3_y' : 'Experience of feeling offended by someone',
# 'nutrition' : 'Malnutrition',
# 'H16_4_y' : 'Relation satisfaction with children',
# 'J1b_1_y' : 'Expense for medical treatment',
# 'H16_2_y' : 'Subjective economic satisfaction',
# 'hospital_no' : 'Unable to receive medical treatment ',
# 'cohab_mean_edu_y' : 'Mean educational year of cohabitants',
# 'G4_3_6_y' : 'Helping spouse for medical purpose', ## 도움 줌
# 'H16_3_y' : 'Relation satisfaction with spouse',
# 'call_child_out' : 'Frequency of phone call with seperated children',
# 'J6b_4_y' : 'Financial debt',
# 'friends_contact' : 'Frequency of contacting with friends',
# 'fac_dist' : 'Distance to community facilities ',
# 'meet_child_out' : 'Frequency of meeting seperated children',
# 'RES_EDU2_y' : 'Educational year',
# 'bmi' : 'BMI',
# 'B5_1_etc_1_y' : 'Number of nursing hospital visits',
# 'B2_3_y' : 'Number of chronic diseases',
# 'B4_1_etc_y' : 'Number of hospital visits',
# 'num_alter' : 'Number of friends',
# 'D8_y_0' : 'Having no religion',
# 'J1b_4_y' : 'Expense for events (e.g., funeral)',
# 'E2_2_1_y' : 'Length of prior work experience',
# 'cohab_mean_age' : 'Mean age of cohabitants',
# 'Q1_y_1': 'Living in a detached house',
# 'J3b_3_13_y' : 'family total income',



# }


X_ex_val_r = X_ex_val.rename(columns=col_dict)
df = X_ex_val_r.sample(n=1000, random_state = 1120)

In [ ]:
shap_values = explainer.shap_values(df, check_additivity=False)

In [ ]:
shap.summary_plot(shap_values, df, max_display=20)

In [ ]:
shap.summary_plot(shap_values, df, plot_type="bar", max_display=20)

In [ ]:
# data_path = '/content/gdrive/My Drive/성은이파이썬/ICT_ML/data/2020_NSOK_preprocessed_2.csv'

# ex_val_data = pd.read_csv(data_path) ## sperater, delimiter 확인
# ex_val_data = ex_val_data.drop('Unnamed: 0', axis=1)
ex_val_data = test_2020

X_ex_val = ex_val_data.drop('suicide', axis=1)
y_ex_val = ex_val_data['suicide']

In [ ]:
# col_dict = {'performance' : 'Physical malperform',
# 'gds15_i' : 'Depression (gds15)',
# 'living_sudo' : 'Living in metropolitan area',
# 'RES_AGE_y' : 'Age',
# 'H16_7_y' : 'Life satisfaction',
# 'H16_4_y' : 'Relation satisfaction with children',
# 'H16_2_y' : 'Subjective economic satisfaction',
# 'nutrition' : 'Malnutrition',
# 'H14_1_3_y' : 'Experience of feeling offended by someone',
# 'J1b_1_y' : 'Expense for medical treatment',
# 'G4_3_6_y' : 'Helping spouse for medical purpose', ## 도움 줌
# 'H16_3_y' : 'Relation satisfaction with spouse',
# 'cohab_mean_edu_y' : 'Mean educational year of cohabitants',
# 'discrimination' : 'Experience of discrimination',
# 'call_child_out' : 'Frequency of phone call with seperated children',
# 'J6b_4_y' : 'Financial debt',
# 'meet_child_out' : 'Frequency of meeting seperated children',
# 'fac_dist' : 'Distance to community facilities ',
# 'hospital_no' : 'Unable to receive medical treatment ',
# 'friends_contact' : 'Frequency of contacting with friends',
# 'bmi' : 'BMI',
# 'B4_1_etc_y' : 'Number of hospital visits',
# 'J1b_4_y' : 'Expense for events (e.g., funeral)',
# 'B2_3_y' : 'Number of chronic diseases',
# 'J3b_3_13_y' : 'family total income',
# 'D8_y_0' : 'Having no religion',
# 'E2_2_1_y' : 'Length of prior work experience',
# 'J6b_2_y' : 'Financial property',
# 'death_prep' : 'Prepared for death',
# 'J3b_3_8_r_y' : 'Total amount of family pension',
# 'num_alter' : 'Number of friends',

# }


X_ex_val = X_ex_val.rename(columns=col_dict)
df = X_ex_val.sample(n=1000, random_state = 1120)

In [ ]:
shap_values = explainer.shap_values(df, check_additivity=False)

In [ ]:
shap.summary_plot(shap_values, df, max_display=20)

In [ ]:
shap.summary_plot(shap_values, df, plot_type="bar", max_display=20)

## ROC curve

In [ ]:
# model_file_path = "/content/gdrive/My Drive/성은이파이썬/ICT_ML/model/model_LR.model"  
# result_lr = joblib.load(model_file_path)

# model_file_path = "/content/gdrive/My Drive/성은이파이썬/ICT_ML/model/model_RF.model"  
# result_RF = joblib.load(model_file_path)

# model_file_path = "/content/gdrive/My Drive/성은이파이썬/ICT_ML/model/model_MLP.model"  
# result_MLP = joblib.load(model_file_path)

# model_file_path = "/content/gdrive/My Drive/성은이파이썬/ICT_ML/model/model_XGB.model"  
# result_XGB = joblib.load(model_file_path)

In [ ]:
# from sklearn import metrics
# import numpy as np
# import matplotlib.pyplot as plt

# y_test_pred  = result.predict(X_test)
# y_test_prob = result.predict_proba(X_test)

# y_ex_val, y_ex_val_pred

# plt.figure(0).clf()

# pred = np.random.rand(1000)
# label = np.random.randint(2, size=1000)
# fpr, tpr, thresh = metrics.roc_curve(label, pred)
# auc = metrics.roc_auc_score(label, pred)
# plt.plot(fpr,tpr,label="data 1, auc="+str(auc))

# pred = np.random.rand(1000)
# label = np.random.randint(2, size=1000)
# fpr, tpr, thresh = metrics.roc_curve(label, pred)
# auc = metrics.roc_auc_score(label, pred)
# plt.plot(fpr,tpr,label="data 2, auc="+str(auc))

# plt.legend(loc=0)